In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14582578204001136002
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 549487290235473912
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]


2023-12-01 18:27:25.992596: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-12-01 18:27:25.992618: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-12-01 18:27:25.992624: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-12-01 18:27:25.992690: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-01 18:27:25.992909: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Concatenate, Conv2D, Conv2DTranspose, \
# GlobalAveragePooling2D, UpSampling2D, LeakyReLU, ReLU, Add, Multiply, Lambda, Dot, BatchNormalization, Activation, \
# ZeroPadding2D, Cropping2D, Cropping1D
# from tensorflow.keras.models import Sequential, Model, load_model

import os
import librosa
import soundfile as sf
import IPython
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd

In [4]:
genre_path_1 = 'data_2/genres_original/classical'
cls_names = sorted([os.path.join(genre_path_1, f) for f in os.listdir(genre_path_1) if os.path.isfile(os.path.join(genre_path_1, f))])

genre_path_2 = 'data_2/genres_original/jazz'
jzz_names = sorted([os.path.join(genre_path_2, f) for f in os.listdir(genre_path_2) if os.path.isfile(os.path.join(genre_path_2, f)) \
                 and 'jazz.00054.wav' not in f])

genre_path_3 = 'data_2/genres_original/country'
ctry_names = sorted([os.path.join(genre_path_3, f) for f in os.listdir(genre_path_3) if os.path.isfile(os.path.join(genre_path_3, f))])

# genre_path_1 = 'data_2/genres_original/blues'
# genre_1 = sorted([os.path.join(genre_path_1, f) for f in os.listdir(genre_path_1) if os.path.isfile(os.path.join(genre_path_1, f))])

# genre_path_1 = 'data_2/genres_original/blues'
# genre_1 = sorted([os.path.join(genre_path_1, f) for f in os.listdir(genre_path_1) if os.path.isfile(os.path.join(genre_path_1, f))])

# genre_path_1 = 'data_2/genres_original/blues'
# genre_1 = sorted([os.path.join(genre_path_1, f) for f in os.listdir(genre_path_1) if os.path.isfile(os.path.join(genre_path_1, f))])

# genre_path_1 = 'data_2/genres_original/blues'
# genre_1 = sorted([os.path.join(genre_path_1, f) for f in os.listdir(genre_path_1) if os.path.isfile(os.path.join(genre_path_1, f))])

# genre_path_1 = 'data_2/genres_original/blues'
# genre_1 = sorted([os.path.join(genre_path_1, f) for f in os.listdir(genre_path_1) if os.path.isfile(os.path.join(genre_path_1, f))])

# print(speeker_names_1[0], speeker_names_2[0], speeker_names_3[0], speeker_names_4[0])

In [5]:
s_rate = 22050
n_fft = 2048
hop_length = 512

def load_and_process_audio(speech_names):
    data = [load_audio(x) for x in speech_names]
    max_t = max([x.shape[1] for x in data])
    data = [pad_zeros(x, max_t) for x in data]
    data = np.array(data)
    # data = np.transpose(data, (0, 2, 1))
    return data

def load_audio(speech_name):
    s, sr = librosa.load(speech_name, sr=s_rate)
    sr = librosa.stft(s, n_fft=n_fft, hop_length=hop_length)
    return np.abs(sr)

def pad(data, T):
    data = [pad_zeros(x, T) for x in data]
    return np.array(data)

def pad_zeros(sr, T):
    return np.pad(sr, ((0, 0), (0, T - sr.shape[1])), mode='constant')

In [6]:
cls = load_and_process_audio(cls_names)
jzz = load_and_process_audio(jzz_names)
ctry = load_and_process_audio(ctry_names)

In [7]:
print(cls.shape, jzz.shape, ctry.shape)

(100, 1025, 1314) (99, 1025, 1313) (100, 1025, 1308)


In [8]:
max_t = max([x[2] for x in [cls.shape, jzz.shape, ctry.shape]])
cls = pad(cls, max_t)

max_t = max([x[2] for x in [cls.shape, jzz.shape, ctry.shape]])
jzz = pad(jzz, max_t)
print(jzz.shape)


max_t = max([x[2] for x in [cls.shape, jzz.shape, ctry.shape]])
ctry = pad(ctry, max_t)


(99, 1025, 1314)


In [9]:
cls = np.transpose(cls, (0, 2, 1))
jzz = np.transpose(jzz, (0, 2, 1))
ctry = np.transpose(ctry, (0, 2, 1))

In [10]:
cls_s = cls[np.random.choice(cls.shape[0], size=50, replace=False)]
jzz_s = jzz[np.random.choice(jzz.shape[0], size=25, replace=False)]
ctry_s = ctry[np.random.choice(ctry.shape[0], size=25, replace=False)]

In [11]:
print(cls_s.shape, jzz_s.shape, ctry_s.shape)

(50, 1314, 1025) (25, 1314, 1025) (25, 1314, 1025)


In [12]:
data = np.concatenate((cls_s, jzz_s, ctry_s), axis=0)
data.shape

(100, 1314, 1025)

In [ ]:
from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
#from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

#Download instance norm. code from the link above.
#Or install keras_contrib using guidelines here: https://github.com/keras-team/keras-contrib 
from instancenormalization import InstanceNormalization 

In [ ]:
c_model_BtoA = load_model('c_model_BtoA.h5')